In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import folium


In [4]:
df = pd.read_csv('dataset/train.csv')
df

,id,Classificação do hotel,Meses da reserva até o check-in,Número de pernoites reservadas,Número de hospedes,Regime de alimentação,Nacionalidade,Forma de Reserva,Já se hospedou anterioremente,Tipo do quarto reservado,Reserva feita por agência de turismo,Reserva feita por empresa,Reserva com Estacionamento,Reserva com Observações,Reserva Cancelada
0,33571,5 estrelas,5,5,2.0,Café da manha e jantar,France,Agência,Não,Amethyst,Sim,Não,Sim,Nenhuma,0
1,82458,4 estrelas,167,3,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
2,94061,4 estrelas,4,3,2.0,Café da manha,Belgium,Agência,Não,Amethyst,Sim,Não,Não,1 a 3,0
3,75196,4 estrelas,13,2,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
4,82940,4 estrelas,4,2,2.0,Café da manha,Spain,Agência,Não,Amethyst,Sim,Não,Não,Nenhuma,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72154,101851,4 estrelas,2,3,2.0,Café da manha,France,Agência,Não,Green Emerald,Sim,Não,Não,1 a 3,0
72155,50800,4 estrelas,4,2,2.0,Sem refeicao,Italy,Agência,Não,Amethyst,Sim,Não,Sim,1 a 3,0
72156,4101,5 estrelas,5,2,2.0,Café da manha e jantar,Spain,Agência,Não,Amethyst,Não,Não,Não,Nenhuma,1
72157,15828,5 estrelas,1,8,2.0,Café da manha e jantar,Spain,Balcão,Não,Pink Sapphire,Sim,Não,Não,Nenhuma,0
